In [55]:
import requests
import pandas as pd
import json
from dateutil import parser

In [56]:
API_KEY = "67939bb0d275fb01694c238a5f6f7da5-3e9ae23411b17b20ead1a6ca001b5680"
ACCOUNT_ID = "101-002-27691953-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [57]:
session = requests.Session()

In [58]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [59]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [60]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [61]:
response = session.get(url, params=None, data=None, headers=None)

In [62]:
response.status_code

200

In [63]:
data = response.json()

In [64]:
instruments_list = data['instruments']

In [65]:
len(instruments_list)

123

In [66]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [67]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [68]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i }

In [69]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [1]:
# -4 -> 0.0001
pow(10, -4)

0.0001

In [70]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [75]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

In [77]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [78]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 candles, 2023-12-12 10:00:00+00:00 2023-12-13 18:00:00+00:00


In [79]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [85]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 candles, 2023-04-25 09:00:00+00:00 2023-12-14 01:00:00+00:00
EUR_USD H4 4000 candles, 2021-05-24 09:00:00+00:00 2023-12-13 22:00:00+00:00
EUR_GBP H1 4000 candles, 2023-04-25 09:00:00+00:00 2023-12-14 01:00:00+00:00
EUR_GBP H4 4000 candles, 2021-05-24 09:00:00+00:00 2023-12-13 22:00:00+00:00
EUR_JPY H1 4000 candles, 2023-04-25 09:00:00+00:00 2023-12-14 01:00:00+00:00
EUR_JPY H4 4000 candles, 2021-05-24 01:00:00+00:00 2023-12-13 22:00:00+00:00
EUR_CHF H1 4000 candles, 2023-04-25 09:00:00+00:00 2023-12-14 01:00:00+00:00
EUR_CHF H4 4000 candles, 2021-05-24 09:00:00+00:00 2023-12-13 22:00:00+00:00
EUR_NZD H1 4000 candles, 2023-04-25 08:00:00+00:00 2023-12-14 01:00:00+00:00
EUR_NZD H4 4000 candles, 2021-05-24 09:00:00+00:00 2023-12-13 22:00:00+00:00
EUR_CAD H1 4000 candles, 2023-04-25 09:00:00+00:00 2023-12-14 01:00:00+00:00
EUR_CAD H4 4000 candles, 2021-05-24 09:00:00+00:00 2023-12-13 22:00:00+00:00
EUR_AUD H1 4000 candles, 2023-04-25 09:00:00+00:00 2023-12-14 01:00:00+00:00